In [1]:
# Uncomment the next line if catboost is not already installed:
!pip install catboost

import pandas as pd
import numpy as np

# Import models from scikit-learn and external libraries
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              ExtraTreesRegressor, AdaBoostRegressor)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor  # CART model
from sklearn.linear_model import (LinearRegression, Lasso, ElasticNet, BayesianRidge, Ridge)
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Mount Google Drive (if running in Colab)
from google.colab import drive
drive.mount('/content/drive')

# Load dataset from the new online location (using the raw URL)
data_url = "https://raw.githubusercontent.com/apownukepcc/cosc1315spring2025/main/final_dataset_with_weather.csv"
data = pd.read_csv(data_url)

# Convert the 'date' column to datetime
data['date'] = pd.to_datetime(data['date'])

# Filter to peak season months (May through August)
peak_season_months = [5, 6, 7, 8]
data = data[data['date'].dt.month.isin(peak_season_months)]

# Define predictors (weather variables) and the three emission load targets
predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
targets = ['Emissions_Load_SO2TONS', 'Emissions_Load_NOXTONS', 'Emissions_Load_COTONS']

# Drop rows with missing values in predictors or any of the target columns
data = data.dropna(subset=predictors + targets)

# Define a comprehensive set of predictive models
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "k-NN": KNeighborsRegressor(n_neighbors=5),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=500, random_state=42),
    "Linear Regression": LinearRegression(),
    "CART": DecisionTreeRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(kernel='rbf'),
    "Extra Trees": ExtraTreesRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(random_state=42, verbose=0),
    "Bayesian Ridge": BayesianRidge(),
    "Ridge": Ridge(random_state=42)
}

# Dictionary to hold overall average relative errors per model for each target across all sources
overall_rel_errors = {target: {model_name: [] for model_name in models.keys()} for target in targets}

# Dictionary to hold summary metrics per source for each target
source_summary = {target: {} for target in targets}

# Loop over each emission load target
for target in targets:
    print("\n" + "="*80)
    print(f"Evaluating predictive models for target: {target}")
    print("="*80)

    # Global predictions table for this target (if needed later)
    all_predictions_table = pd.DataFrame()

    # Process predictions for each unique powerplant in the "Source" column
    for source in data['Source'].unique():
        data_source = data[data['Source'] == source].copy()

        # Check if there is enough data for meaningful predictions
        if data_source.shape[0] < 10:
            print(f"\nPowerplant: {source} -- Not enough data, skipping...")
            continue

        # Split features and target for the current source
        X = data_source[predictors]
        y = data_source[target]

        # Split into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        y_test_array = y_test.values

        # Dictionaries to hold predictions and performance metrics for this powerplant
        predictions = {}
        performance_metrics = {}
        source_metrics = {model_name: {"RMSE": [], "MAE": [], "R2": [], "MAPE": []} for model_name in models.keys()}

        # Train each model and compute predictions and metrics
        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            predictions[model_name] = y_pred

            rmse = np.sqrt(mean_squared_error(y_test_array, y_pred))
            mae  = mean_absolute_error(y_test_array, y_pred)
            r2   = r2_score(y_test_array, y_pred)
            mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100

            performance_metrics[model_name] = {"RMSE": rmse, "MAE": mae, "R2": r2, "MAPE": mape}
            source_metrics[model_name]["RMSE"].append(rmse)
            source_metrics[model_name]["MAE"].append(mae)
            source_metrics[model_name]["R2"].append(r2)
            source_metrics[model_name]["MAPE"].append(mape)

            # Collect relative error for overall summary across sources
            rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
            overall_rel_errors[target][model_name].append(rel_error)

        # Build a predictions table for the current powerplant (if needed for record)
        pred_table = pd.DataFrame({
            "Date": data_source.loc[X_test.index, 'date'].values,
            "Actual": y_test_array
        })
        for model_name, y_pred in predictions.items():
            pred_table[model_name] = y_pred
            residual = y_pred - y_test_array
            pred_table[model_name + " Residual"] = residual
            pred_table[model_name + " Relative Error (%)"] = (np.abs(residual) / y_test_array) * 100
        pred_table["Source"] = source
        all_predictions_table = pd.concat([all_predictions_table, pred_table], ignore_index=True)

        # Create summary table for this powerplant (average metrics per model)
        summary_data = []
        for model_name, metrics in source_metrics.items():
            avg_rmse = np.mean(metrics["RMSE"]) if metrics["RMSE"] else np.nan
            avg_mae  = np.mean(metrics["MAE"])  if metrics["MAE"]  else np.nan
            avg_r2   = np.mean(metrics["R2"])   if metrics["R2"]   else np.nan
            avg_mape = np.mean(metrics["MAPE"])  if metrics["MAPE"] else np.nan
            summary_data.append({
                "Model": model_name,
                "Avg_RMSE": avg_rmse,
                "Avg_MAE": avg_mae,
                "Avg_R2": avg_r2,
                "Avg_MAPE": avg_mape
            })
        summary_df = pd.DataFrame(summary_data)
        source_summary[target][source] = summary_df.copy()

        # Print performance metrics for this powerplant
        print(f"\nPerformance summary for powerplant: {source} (target: {target})")
        print(summary_df.sort_values(by="Avg_RMSE", ascending=True).to_string(index=False))

        # Also print sorted best/worst per metric for this source
        metrics_to_sort = {"Avg_RMSE": "min", "Avg_MAE": "min", "Avg_MAPE": "min", "Avg_R2": "max"}
        for metric, sort_order in metrics_to_sort.items():
            sorted_df = summary_df.sort_values(by=metric, ascending=(sort_order=="min"))
            best_model = sorted_df.iloc[0]["Model"]
            worst_model = sorted_df.iloc[-1]["Model"]
            print(f"\nFor powerplant {source} (target: {target}) sorted by {metric}:")
            print(sorted_df[["Model", metric]].to_string(index=False))
            print(f"Best {metric}: {best_model}   |   Worst {metric}: {worst_model}")

    # After processing all sources for the current target, print overall average relative errors.
    print("\n" + "-"*80)
    print(f"Overall Average Relative Error (%) for each model across all powerplants (target: {target}):")
    overall_summary = []
    for model_name, errors in overall_rel_errors[target].items():
        if errors:
            avg_rel_error = np.mean(errors)
            overall_summary.append({"Model": model_name, "Overall Avg Rel Error (%)": avg_rel_error})
            print(f"  {model_name}: {avg_rel_error:.2f}%")
        else:
            overall_summary.append({"Model": model_name, "Overall Avg Rel Error (%)": np.nan})
            print(f"  {model_name}: No data available.")
    overall_summary_df = pd.DataFrame(overall_summary)
    overall_summary_df = overall_summary_df.sort_values(by="Overall Avg Rel Error (%)", ascending=True)
    print("\nOverall summary (sorted by relative error):")
    print(overall_summary_df.to_string(index=False))

    # Optionally, you could save the predictions table for each target to CSV on Google Drive.
    # csv_path = f'/content/drive/My Drive/final_predictions_{target}.csv'
    # all_predictions_table.to_csv(csv_path, index=False)
    # print(f"\nGlobal predictions table for {target} saved to {csv_path}")

# End of code. Only summary tables have been printed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.5 MB/s eta 0:00:00
Mounted at /content/drive

Evaluating predictive models for target: Emissions_Load_SO2TONS
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000889 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 257
[LightGBM] [Info] Number of data points in the train set: 181, number of used features: 7
[LightGBM] [Info] Start training from score 0.000036
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e521

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 259
[LightGBM] [Info] Number of data points in the train set: 188, number of used features: 7
[LightGBM] [Info] Start training from score 0.000141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e5212642>:110: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_array - y_pred) / y_test_array)) * 100
<ipython-input-1-c452e5212642>:119: RuntimeWarning: divide by zero encountered in divide
  rel_error = np.mean((np.abs(y_pred - y_test_array) / y_test_array) * 100)
<ipython-input-1-c452e521


Performance summary for powerplant: LAKE-3 (target: Emissions_Load_COTONS)
            Model  Avg_RMSE   Avg_MAE        Avg_R2  Avg_MAPE
            Ridge  0.000062  0.000049  1.192377e-01       inf
   Bayesian Ridge  0.000062  0.000049  1.191307e-01       inf
Linear Regression  0.000062  0.000049  1.191238e-01       inf
         CatBoost  0.000064  0.000049  7.177596e-02       inf
    Random Forest  0.000065  0.000051  4.725370e-02       inf
         AdaBoost  0.000065  0.000051  3.992760e-02       inf
Gradient Boosting  0.000065  0.000051  3.688177e-02       inf
         LightGBM  0.000065  0.000052  1.947655e-02       inf
            Lasso  0.000067  0.000054 -2.936580e-02       inf
       ElasticNet  0.000067  0.000054 -2.936580e-02       inf
          XGBoost  0.000067  0.000054 -2.936580e-02       inf
      Extra Trees  0.000068  0.000053 -5.337714e-02       inf
             k-NN  0.000069  0.000054 -9.595207e-02       inf
             CART  0.000102  0.000085 -1.381821e+00     